In [45]:
%matplotlib inline
import matplotlib.animation
import matplotlib.pyplot as plt
from IPython.display import HTML
import json
import numpy as np
import pandas as pd

# Load the data

In [46]:
with open("data/unlabelled/csi/clap/rx1_tx1/1_clap.csv") as f:
    data = pd.read_csv(f, header=None).transpose()
    data.columns = data.iloc[0]
    data = data.iloc[1:]

data.head()

,timestamp,amplitude,phase
1,1698153140.0,-0.003162,-1.815357
2,1698153140.0,-0.002926,-9.610022
3,1698153140.0,-0.003282,3.69998
4,1698153140.0,-0.003283,0.41413
5,1698153140.0,-0.003103,9.832106


In [47]:
print(data.timestamp.unique())

[1698153140.0 1698153141.0 1698153142.0 1698153143.0 1698153144.0
 1698153145.0]


In [58]:
with open("data/unlabelled/camera/joints/front_sit_stand.csv") as f:
    data = pd.read_csv(f, header=0)
    data = data.rename(columns={"x-axis": "x", "y-axis": "y", "z-axis": "z", "joint_names": "joint_name"})

data.head()

,frame_id,timestamp_usec,joint_name,x,y,z
0,0,268566,PELVIS,-35.965904,121.446609,2231.891113
1,0,268566,SPINE_NAVEL,-46.754833,-36.169590,2264.973877
2,0,268566,SPINE_CHEST,-54.007175,-164.899796,2259.733398
3,0,268566,NECK,-55.790428,-348.966309,2191.183594
4,0,268566,CLAVICLE_LEFT,-25.078732,-319.862854,2204.463135


In [59]:
joint_names = data["joint_name"].unique()
joint_names.size

32

In [60]:
frame_ids = data["frame_id"].unique()
frame_ids.size

2520

In [61]:
np.max(frame_ids)

2519

In [62]:
# Very inneficient way
# import time
# a = time.time()
# struc_data = np.zeros((frame_ids.size, joint_names.size, 3))
# for id in frame_ids:
#     frame = data[data["frame_id"] == id]
#     for i, joint in enumerate(joint_names):
#         struc_data[id, i, :] = frame[frame["joint_names"] == joint][["x", "y", "z"]].values
# print(time.time() - a)   # Something like 18 seconds
#
# Even more unefficient way
# a = time.time()
# struc_data = np.zeros((frame_ids.size, joint_names.size, 3))
# for id in frame_ids:
#     for i, joint in enumerate(joint_names):
#         struc_data[id, i, :] = data[(data["joint_names"] == joint) & (data["frame_id"] == id)][["x", "y", "z"]].values
# print(time.time() - a)  # Something like 100 seconds

In [63]:
# Create a multi-index using 'frame_id' and 'joint_names'
data.set_index(['frame_id', 'joint_name'], inplace=True)

# Sort the index to ensure the data is in the correct order
data.sort_index(inplace=True)

# Convert the DataFrame to a NumPy array and reshape it
struc_data = data[['x', 'y', 'z']].values.reshape((frame_ids.size, joint_names.size, 3))